In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp

In [2]:
class Activation:
    """
    relates nueral excitation and muscle activation.  this is a parent class without methods implemented.
    to use an activation model, use one of the child classes below that either returns adot (the derivative
    of activation) or a itelf (activation) as a function of u (neural input) and a (previous activation)

    Parameters
        u          (float):   unitless excitation value b/w 0-1
        a          (float):   unitless activation value b/w 0-1
        tau_act    (float):   activation timescale 10 ms
        tau_deact  (float):   deactivation timescale 40 ms
    """
    def __init__(self, u, tau_act = 0.01, tau_deact = 0.04):
        self.u = u
        self.tau_act = tau_act
        self.tau_deact = tau_deact
               
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        # maybe give finite differences as adot
        raise Exception('Method implemented in child class')
        
    def a(self, a):
        raise Exception('Method implemented in child class')

In [3]:
class Thelen(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        if self.u >= a:
            adot = (self.u - a) / self.tau_act
        else:
            adot = (self.u - a) / self.tau_deact
        return adot

In [4]:
class Winters(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        if self.u > a:
            tau = self.tau_act * (0.5 + 1.5*a)
        else:
            tau = self.tau_deact / (0.5 + 1.5*a)
        adot = (self.u - a) / tau
        return adot

In [5]:
class Zajac(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        self.beta = self.tau_act / self.tau_deact
        
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        adot = 1 / self.tau_act * ( self.u - a * (self.beta + (1+self.beta) * self.u) )
        return adot

In [6]:
class Manal(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        self.uo = 0.3085 - self.tau_act * np.cos( np.pi / 4 )
        self.ao = 0.3085 + self.tau_act * np.sin( np.pi / 4 )
        self.m = (self.ao-1) / (self.uo-1)
        self.c = 1 - self.m
        self.beta = (np.exp(self.ao / self.tau_deact) - 1) / self.uo
        
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        if self.u <= self.uo:
            adot = self.tau_deact * np.log( self.beta * self.u + 1 )
        elif self.u > self.uo:
            adot = self.m * self.u + self.c
        return adot

In [7]:
class Chadwick(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        adot = ( self.u - a ) * (self.u / self.tau_act + (1 - self.u) / self.tau_deact )
        return adot

In [8]:
class Nagano(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        self.t2 = 1 / self.tau_deact
        self.t1 = 1 / (self.tau_act - self.t2)
        
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        adot = (self.u - a) * (self.t1 * self.u + self.t2)
        return adot

In [9]:
class Rengifo(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        
    def __call__(self, t, a):
        return self.rhs(t, a)
    
    def rhs(self, t, a):
        if self.u >= a:
            adot = -a / self.tau_act + self.u / self.tau_act
        elif a > self.u:
            adot = -a / self.tau_deact + self.u / self.tau_deact
        return adot

In [10]:
class AModel(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        self.A = -2.0
    
    def a(self):
        a = (np.exp(self.tau_act * self.u) - 1) / (np.exp(self.tau_act) - 1)
        return a

In [11]:
class Cavallaro(Activation):
    def __init__(self, u, **kwargs):
        super().__init__(u, **kwargs)
        self.A = -2.0
    
    def a(self):
        a = (self.tau_act**self.u - 1) / (self.tau_act - 1)
        return a